In [2]:
pip install transformers

     -------------------------------------- 43.7/43.7 kB 711.0 kB/s eta 0:00:00
   ---------------------------------------- 9.4/9.4 MB 6.1 MB/s eta 0:00:00
   ---------------------------------------- 417.5/417.5 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 54.0/54.0 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 286.2/286.2 kB 5.9 MB/s eta 0:00:00
   ---------------------------------------- 2.2/2.2 MB 7.4 MB/s eta 0:00:00
   ---------------------------------------- 177.6/177.6 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4
  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.7.4
    Uninstalling fsspec-0.7.4:
      Successfully uninstalled fsspec-0.7.4



DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.1.4 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.4 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install tensorflow

  Using cached numpy-1.18.5-cp38-cp38-win_amd64.whl.metadata (2.1 kB)
  Using cached scipy-1.4.1-cp38-cp38-win_amd64.whl.metadata (2.0 kB)
Using cached scipy-1.4.1-cp38-cp38-win_amd64.whl (31.0 MB)
Using cached numpy-1.18.5-cp38-cp38-win_amd64.whl (12.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


  You can safely remove it manually.
ERROR: Exception:
Traceback (most recent call last):
  File "C:\python\lib\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "C:\python\lib\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "C:\python\lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "C:\python\lib\site-packages\pip\_internal\commands\install.py", line 483, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "C:\python\lib\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(self._dist.version)
  File "C:\python\lib\site-packages\pip\_vendor\packaging\version.py", line 56, in parse
    return Version(version)
  File "C:\python\lib\site-packages\pip\_vendor\packaging\version.py", line 202, in __init__
    raise 

In [2]:
# 필요한 라이브러리 임포트
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
from tqdm import tqdm
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import tensorflow as tf

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles, attrs):
    attrs_content = []
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    original_html = requests.get(url, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver, 'href')
    return url

# 기사 크롤링 및 요약 함수
def summarize_articles(company_name, start_page=1, end_page=300):   #### 몇페이지까지 크롤링할건지 정하기
    # naver url 생성
    url = makeUrl(company_name, start_page, end_page)

    # 뉴스 크롤러 실행
    news_titles = []
    news_contents = []
    final_urls = []
    for i in url:
        urls = articles_crawler(i)
        for link in urls:
            if "news.naver.com" in link:
                final_urls.append(link)

    # 뉴스 내용 크롤링
    for i in tqdm(final_urls, desc=f"크롤링 {company_name}"):
        # 각 기사 html get하기
        news = requests.get(i, headers=headers)
        news_html = BeautifulSoup(news.text, "html.parser")

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title is None:
            title = news_html.select_one("#content > div.end_ct > div > h2")
        
        # 뉴스 본문 가져오기
        content = news_html.select("article#dic_area")
        if not content:
            content = news_html.select("#articeBody")

        # 기사 텍스트만 가져오기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        if title and content:
            news_titles.append(title)
            news_contents.append(content)

    # 크롤링한 기사가 없으면 빈 문자열 반환
    if not news_contents:
        return ""

    # 크롤링한 기사 내용을 하나의 텍스트로 합치기
    combined_text = " ".join(news_contents)

    # KoBART 토크나이저와 모델 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')
    model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')

    # 입력 텍스트의 길이 제한 설정
    max_input_length = 1024

    # 입력 텍스트 길이 조정
    if len(combined_text) > max_input_length:
        combined_text = combined_text[:max_input_length]

    # 입력 텍스트 토큰화
    inputs = tokenizer(combined_text, max_length=max_input_length, return_tensors='pt', truncation=True)

    # 요약 생성
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=80, length_penalty=2.0, num_beams=4, early_stopping=True)

    # 요약 결과 디코딩
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# 엑셀 파일에서 데이터 불러오기
file_path = './전체기업.csv'  # 엑셀 파일 경로
df = pd.read_csv(file_path)

# 뉴스 요약 컬럼 생성
df['뉴스요약'] = ''

# 기업명별로 뉴스 요약 생성하여 추가
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="전체 진행"):
    company_name = row['corp_name']
    summary = summarize_articles(company_name)
    df.at[index, '뉴스요약'] = summary

# 최종 결과를 엑셀 파일로 저장
output_path = './기업별_뉴스요약_결과.csv'  # 결과 파일 경로
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("모든 기업에 대한 뉴스 요약이 완료되었습니다.")


ModuleNotFoundError: No module named 'tensorflow.tsl'

In [ ]:
df

In [9]:
df['뉴스요약'][1]

'·\'\'\'\'→\'\' 한 단계 상향...역대 가장 높아"2년간 의미있는 수준 수익성 개선 시현"\n\n\n\n\n[이천=뉴시스] 김종택 기자 = SK하이닉스가 올해 2분기(4~6월) 올해 2분기(4~6월) 역대 최대 분기 매출을 달성했다. 영업이익도 역대 3번째로 많은 큰 큰 큰 큰 큰 큰 큰 큰 큰 큰 큰으로, 6년 만에 5조원대의 이익을 냈다. SK하이닉스는 올해 2분기 매출 16조4233억원, 영업이익 5조4685억원의억원의 실적을 기록했다고 25일 밝혔다. 매출은 전년 7조3059억원보다 124.8% 증가했고, 영업손익도 전년 2조8821억원 적자 대비 흑자 전환에 성공했다. 사진은'

In [5]:
# PyTorch 설치 (CPU 버전)
!pip install torch


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 198.1/198.1 MB 5.4 MB/s eta 0:00:00


In [6]:
# PyTorch 설치 (CUDA를 지원하는 GPU 버전, NVIDIA GPU 사용 시)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://download.pytorch.org/whl/cu117
     ---------------------------------------- 4.9/4.9 MB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 2.0 MB/s eta 0:00:00
     --------------------------------         1.9/2.3 GB 7.8 MB/s eta 0:00:53


ERROR: Exception:
Traceback (most recent call last):
  File "C:\python\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\python\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\python\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\python\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "C:\python\lib\http\client.py", line 454, in read
    n = self.readinto(b)
  File "C:\python\lib\http\client.py", line 498, in readinto
    n = self.fp.readinto(b)
  File "C:\python\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "C:\python\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\python\lib\ssl.py", line 1099, in r

In [7]:
import torch

## 네이버 코드 보고 데이터 생성 코드 다시 짜봄.

In [12]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


In [13]:
# 필요한 라이브러리 임포트
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
from tqdm import tqdm
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import numpy as np
#엑셀 파일에서 데이터 불러오기
file_path = './기업명_리스트_시범2.csv'  # 엑셀 파일 경로
df = pd.read_csv(file_path)

# 뉴스 요약 컬럼 생성
df['뉴스요약'] = ''

# 기업명별로 뉴스 요약 생성하여 추가
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="전체 진행"):
    
    #####뉴스크롤링 시작#####

    #검색어 입력
    search = row['기업명']
    #검색 시작할 페이지 입력
    page = 1
    #검색 종료할 페이지 입력
    page2 = 5


    # naver url 생성
    url = makeUrl(search,page,page2)

    #뉴스 크롤러 실행
    news_titles = []
    news_url =[]
    news_contents =[]
    news_dates = []
    for i in url:
        url = articles_crawler(url)
        news_url.append(url)


    #제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
    def makeList(newlist, content):
        for i in content:
            for j in i:
                newlist.append(j)
        return newlist

    
    #제목, 링크, 내용 담을 리스트 생성
    news_url_1 = []

    #1차원 리스트로 만들기(내용 제외)
    makeList(news_url_1,news_url)

    #NAVER 뉴스만 남기기
    final_urls = []
    for i in tqdm(range(len(news_url_1))):
        if "news.naver.com" in news_url_1[i]:
            final_urls.append(news_url_1[i])
        else:
            pass

    # 뉴스 내용 크롤링

    for i in tqdm(final_urls):
        #각 기사 html get하기
        news = requests.get(i,headers=headers)
        news_html = BeautifulSoup(news.text,"html.parser")

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")
    
        # 뉴스 본문 가져오기
        content = news_html.select("article#dic_area")
        if content == []:
            content = news_html.select("#articeBody")

        # 기사 텍스트만 가져오기
        #list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        try:
            html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
            news_date = html_date.attrs['data-date-time']
        except AttributeError:
            news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
            news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
        # 날짜 가져오기
        news_dates.append(news_date)

#print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
#print("\n[뉴스 제목]")
#print(news_titles)
#print("\n[뉴스 링크]")
#print(final_urls)
#print("\n[뉴스 내용]")
#print(news_contents)

#print('news_title: ',len(news_titles))
#print('news_url: ',len(final_urls))
#print('news_contents: ',len(news_contents))
#print('news_dates: ',len(news_dates))

        ###데이터 프레임으로 만들기###

        #데이터 프레임 만들기
        news_df = pd.DataFrame({'corp_name':search,'content':news_contents})

        #중복 행 지우기
        news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
        print("중복 제거 후 행 개수: ",len(news_df))
    

 # 최종 결과를 엑셀 파일로 저장
output_path = './final.csv'  # 결과 파일 경로
news_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("모든 기업에 대한 뉴스 요약이 완료되었습니다.")

전체 진행:   0%|                                                                                 | 0/2 [00:00<?, ?it/s]

생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&start=41']



100%|██████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<?, ?it/s]

  2%|█▋                                                                                 | 1/49 [00:00<00:26,  1.81it/s]

중복 제거 후 행 개수:  1



  4%|███▍                                                                               | 2/49 [00:00<00:23,  1.98it/s]

...

중복 제거 후 행 개수:  32



 97%|███████████████████████████████████████████████████████████████████████████████▋  | 35/36 [00:17<00:00,  1.88it/s]

중복 제거 후 행 개수:  33



전체 진행: 100%|█████████████████████████████████████████████████████████████████████████| 2/2 [00:51<00:00, 25.67s/it]

중복 제거 후 행 개수:  34
모든 기업에 대한 뉴스 요약이 완료되었습니다.


In [18]:
news_df['content'][1]

'[\n\n\n\n\n [사진출처=연합뉴스]지난 4월 미국 정부로부터 약 9조원의 보조금을 확정지은 삼성전자에 이어 SK하이닉스 역시 반도체 보조금을 지원받는다.6일(현지시간) 미 상무부는 SK하이닉스와 HBM 고급 패키징 제조, 연구개발(R&amp;D) 시설 설립을 위해 최대 4억5000만달러(6200억원)의 직접보조금과 5억달러(6900억원)의 대출을 지원하는 내용의 예비거래각서(PMT)를 체결했다고 밝혔다.아울러 미 상무부는 SK하이닉스가 미국에서 투자하는 금액의 최대 25%까지 세액 공제 혜택을 제공해 준다고 밝혔다. 최종 지원금은 미 상무부 반도체법 재정 인센티브 세부 지원계획(NOFO)의 절차에 따라 결정될 예정이다.이로써 SK하이닉스는 미국 인디애나주 웨스트라피엣에 반도체 패키징 생산기지 건설을 시작하고 미국 시장 공략에 속도를 낸다는 방침이다.곽노정 SK하이닉스 사장은 “미국 상무부의 지원에 깊은 감사의 뜻을 전하며 이 프로젝트가 완전히 실현되는데 협력할 수 있어 기쁘다”며 “AI 기술을 위한 새로운 허브를 구축하고 인디애나에서 숙련된 일자리를 창출하며 글로벌 반도체 산업을 위한 강력하고 회복력 있는 공급망을 구축하는 데 기여할 수 있길 기대한다”고 말했다.투자금 대비 보조금 비율…삼성 14.2%·SK 11.6%이번 보조금은 조 바이든 대통령이 지난 2022년 제정한 ‘반도체 및 과학법(Chips and Science Act)’에 따른 것이다.미국은 반도체 기업의 미국 내 설비투자를 장려하기 위해 미국에 공장을 짓는 기업에 반도체 생산 보조금으로 총 390억달러(약 52조3000억원), R&amp;D 지원금 132억달러(18조원) 등 5년간 총 527억달러(70조7000억원)을 지원하기로 했다.칩스법에 수혜 대상에는 세계 5대 칩 제조업체가 모두 포함됐다. 삼성전자 64억달러(8조8000억원)을 포함해 ▲TSMC(66억달러·9조) ▲인텔(85억달러·11조7000억원) ▲마이크론테크놀로지(61억달러·8조3000억원) ▲SK하이닉스(4억5000만달러